## 第7章: 単語ベクトル
単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して，以下の処理を行うプログラムを作成せよ．

#### 60. 単語ベクトルの読み込みと表示
Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

指定の学習済み単語ベクトルをダウンロードします。

In [ ]:
%load_ext lab_black

In [ ]:
# FILE_ID = "0B7XkCwpI5KDYNlNUTTlSS21pQmM"
# FILE_NAME = "GoogleNews-vectors-negative300.bin.gz"
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt

# 下記からダウンロードできます。
# https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

自然言語処理のさまざまなタスクで利用されるGensimを用いて、単語ベクトルを読み込みます。

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(
    "./GoogleNews-vectors-negative300.bin.gz", binary=True
)

読み込んだ後は、ベクトル化したい単語を指定するだけで簡単に単語ベクトルを得ることができます。

In [ ]:
model["United_States"]

#### 61. 単語の類似度
“United States”と”U.S.”のコサイン類似度を計算せよ．

ここではsimilarityメソッドを利用します。2単語を指定すると、単語間のコサイン類似度を計算することができます。

In [ ]:
model.similarity("United_States", "U.S.")

#### 62. 類似度の高い単語10件
“United States”とコサイン類似度が高い10語と，その類似度を出力せよ．

ここではmost_similarメソッドを利用します。単語を指定すると、topnまでの類似度上位単語とその類似度を取得することができます。



In [ ]:
model.most_similar("United_States", topn=10)

####　63. 加法構成性によるアナロジー
“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

前問でも利用したmost_similarメソッドは、足すベクトルと引くベクトルをそれぞれ指定した上で、計算後のベクトルと類似度が高い単語を取得することができます。
ここでは、問題文の指示に従い、Spain - Madrid + Athensのベクトルと類似度の高い単語を表示していますが、期待通りGreeceが1位に登場しています。

In [ ]:
vec = model["Spain"] - model["madrid"] + model["Athens"]
model.most_similar(positive=["Spain", "Athens"], negative=["Madrid"], topn=10)

#### 64. アナロジーデータでの実験
単語アナロジーの評価データをダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

指定のデータをダウンロードします。

In [ ]:
!wget http://download.tensorflow.org/data/questions-words.txt

In [ ]:
# 先頭10行の確認
!head -10 questions-words.txt

1行ずつ読込み、指定の単語と類似度を計算した上で整形したデータを出力します。

In [ ]:
with open("./questions-words.txt", "r") as f1, open(
    "./questions-words-add.txt", "w"
) as f2:
    for line in f1:  # f1から1行ずつ読込み、求めた単語と類似度を追加してf2に書込む
        line = line.split()
        if line[0] == ":":
            category = line[1]
        else:
            word, cos = model.most_similar(
                positive=[line[1], line[2]], negative=[line[0]], topn=1
            )[0]
            f2.write(" ".join([category] + line + [word, str(cos) + "\n"]))

In [ ]:
!head -10 questions-words-add.txt

#### 65. アナロジータスクでの正解率
64の実行結果を用い，意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ．

対応するカテゴリごとにそれぞれ計算します。

In [ ]:
with open("./questions-words-add.txt", "r") as f:
    sem_cnt = 0
    sem_cor = 0
    syn_cnt = 0
    syn_cor = 0
    for line in f:
        line = line.split()
        if not line[0].startswith("gram"):
            sem_cnt += 1
            if line[4] == line[5]:
                sem_cor += 1
        else:
            syn_cnt += 1
            if line[4] == line[5]:
                syn_cor += 1

print(f"意味的アナロジー正解率: {sem_cor/sem_cnt:.3f}")
print(f"文法的アナロジー正解率: {syn_cor/syn_cnt:.3f}")

#### 66. WordSimilarity-353での評価
The WordSimilarity-353 Test Collectionの評価データをダウンロードし，単語ベクトルにより計算される類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

このデータは、単語のペアに対して人間が評価した類似度が付与されています。
それぞれのペアに対して単語ベクトルの類似度を計算し、両者のスピアマン順位相関係数を計算します。

In [ ]:
!wget http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
!unzip wordsim353.zip

In [ ]:
!head -10 './combined.csv'

In [ ]:
# 国名の取得
countries = set()
with open("./questions-words-add.txt") as f:
    for line in f:
        line = line.split()
        if line[0] in ["capital-common-countries", "capital-world"]:
            countries.add(line[2])
        elif line[0] in ["currency", "gram6-nationality-adjective"]:
            countries.add(line[1])
countries = list(countries)

# 単語ベクトルの取得
countries_vec = [model[country] for country in countries]

#### 67. k-meansクラスタリング
国名に関する単語ベクトルを抽出し，k-meansクラスタリングをクラスタ数k=5として実行せよ．

適当な国名リストの取得元が見つからなかったため、単語アナロジーの評価データから収集しています。

In [ ]:
# 国名の取得
countries = set()
with open("./questions-words-add.txt") as f:
    for line in f:
        line = line.split()
        if line[0] in ["capital-common-countries", "capital-world"]:
            countries.add(line[2])
        elif line[0] in ["currency", "gram6-nationality-adjective"]:
            countries.add(line[1])
countries = list(countries)

# 単語ベクトルの取得
countries_vec = [model[country] for country in countries]

In [ ]:
from sklearn.cluster import KMeans

# k-meansクラスタリング
kmeans = KMeans(n_clusters=5)
kmeans.fit(countries_vec)
for i in range(5):
    cluster = np.where(kmeans.labels_ == i)[0]
    print("cluster", i)
    print(", ".join([countries[k] for k in cluster]))

#### 68. Ward法によるクラスタリング
国名に関する単語ベクトルに対し，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# k-meansクラスタリング
kmeans = KMeans(n_clusters=5)
kmeans.fit(countries_vec)
for i in range(5):
    cluster = np.where(kmeans.labels_ == i)[0]
    print("cluster", i)
    print(", ".join([countries[k] for k in cluster]))

#### 69. t-SNEによる可視化
国名に関する単語ベクトルのベクトル空間をt-SNEで可視化せよ．

t-SNEで単語ベクトルを2次元に圧縮し、散布図で可視化します。

In [ ]:
# !pip install bhtsne

In [ ]:
# import bhtsne

# embedded = bhtsne.tsne(np.array(countries_vec).astype(np.float64), dimensions=2, rand_seed=123)
# plt.figure(figsize=(10, 10))
# plt.scatter(np.array(embedded).T[0], np.array(embedded).T[1])
# for (x, y), name in zip(embedded, countries):
#     plt.annotate(name, (x, y))
# plt.show()